# Notebook 6: Subsampling informado de MovieLens

En este notebook generamos subconjuntos del dataset de ratings basados en criterios de actividad de usuarios e ítems, para evaluar el impacto del tamaño y la distribución de los datos en los modelos de recomendación y en la eficiencia energética.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

## Cargar datos procesados

In [2]:
data_path = Path("../data/processed")
ratings = pd.read_csv(data_path / "ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Submuestreo informado por actividad

Seleccionaremos usuarios e ítems con mayor número de interacciones.

In [3]:
def top_users(df, fraction):
    counts = df['userId'].value_counts()
    top_n = int(len(counts) * fraction)
    top_users_ids = counts.nlargest(top_n).index
    return df[df['userId'].isin(top_users_ids)].reset_index(drop=True)

def top_items(df, fraction):
    counts = df['movieId'].value_counts()
    top_n = int(len(counts) * fraction)
    top_items_ids = counts.nlargest(top_n).index
    return df[df['movieId'].isin(top_items_ids)].reset_index(drop=True)

## Crear subconjuntos

Generaremos datasets con 10%, 25%, 50% y 75% de los usuarios/ítems más activos.

In [4]:
fractions = [0.1, 0.25, 0.5, 0.75]
subsampled_path = data_path / "subsampled_informed"
subsampled_path.mkdir(exist_ok=True)

for frac in fractions:
    subset_users = top_users(ratings, frac)
    subset_items = top_items(ratings, frac)
    
    subset_users.to_csv(subsampled_path / f"ratings_top_users_{int(frac*100)}.csv", index=False)
    subset_items.to_csv(subsampled_path / f"ratings_top_items_{int(frac*100)}.csv", index=False)
    
    print(f"Subconjunto {int(frac*100)}% guardado: {len(subset_users)} filas (usuarios), {len(subset_items)} filas (ítems)")

Subconjunto 10% guardado: 381007 filas (usuarios), 443904 filas (ítems)
Subconjunto 25% guardado: 640906 filas (usuarios), 723075 filas (ítems)
Subconjunto 50% guardado: 854612 filas (usuarios), 923889 filas (ítems)
Subconjunto 75% guardado: 954369 filas (usuarios), 988678 filas (ítems)


## Comentarios finales

Estos subconjuntos informados permitirán analizar:
- Cómo afecta la distribución de los datos al rendimiento de los modelos.
- El consumo energético en escenarios más realistas.
- Comparación con submuestreo aleatorio.